<a href="https://colab.research.google.com/github/yip-i/asr_capstone/blob/main/create_language_model_xlsr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*italicized text*# New Section

Edited off of Patrick von Paten's tutorial

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 553.3 kB 1.8 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.0.0-cp38-cp38-linux_x86_64.whl size=3268943 sha256=eec3c6d76868268cd56ed965dada726545fadb5ce106ecb6ac25cf375659353e
  Stored in directory: /tmp/pip-ephem-wheel-cache-670lfa_8/wheels/ff/08/4e/a3ddc0e786e0f3c1fcd2e7a82c4324c02fc3ae2638471406d2
Successfully built kenlm
ERROR: Invalid requirement: 'datasets=2.0.0'
Hint: = is not a valid operator. Did you mean == ?
ERROR: unknown command "transformers==4.18.0"


In [ ]:
!pip install pyctcdecode==0.3.0
!pip install datasets==2.7.0
!pip install transformers==4.23.1
!pip install jiwer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.8 MB/s eta 0:00:00


Let's load a small excerpt of the [Librispeech dataset](https://huggingface.co/datasets/librispeech_asr) to demonstrate Wav2Vec2's speech transcription capabilities.

We can see that the `logits` correspond to a sequence of 624 vectors each having 32 entries. Each of the 32 entries thereby stands for the logit probability of one of the 32 possible output characters of the model:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
target_lang="en"  # change to your target lang
model_name = "yip-i/torgo_xlsr_finetune-F04-2"
speaker = "F04"

Let's download the data.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("europarl_bilingual", lang1="en", lang2="sv", split="train")
dataset

Generating train split:   0%|          | 0/1892723 [00:00<?, ? examples/s]

Dataset europarl_bilingual downloaded and prepared to /root/.cache/huggingface/datasets/europarl_bilingual/en-sv-lang1=en,lang2=sv/8.0.0/2ab0200e7729616bfd4a4df6bfb29b31746ceb5a59f8c75c02ca35e1ebead950. Subsequent calls will reuse this data.


Dataset({
    features: ['translation'],
    num_rows: 1892723
})

We see that the data is quite large - it has over a million translations. Since it's only text data, it should be relatively easy to process though.

Next, let's look at how the data was preprocessed when training the fine-tuned *XLS-R* checkpoint in Swedish. Looking at the [`run.sh` file](https://huggingface.co/hf-test/xls-r-300m-sv/blob/main/run.sh), we can see that the following characters were removed from the official transcriptions:

In [ ]:
chars_to_ignore_regex = '[,?.!\-\;\:\"“%‘”�—’…–]'  # change to the ignored characters of your fine-tuned model

Let's do the same here so that the alphabet of our language model matches the one of the fine-tuned acoustic checkpoints.

We can write a single map function to extract the Swedish text and process it right away.

In [ ]:
import re

def extract_text(batch):
  text = batch["translation"][target_lang]
  batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
  return batch

Let's apply the `.map()` function. This should take roughly 5 minutes.

In [ ]:
dataset = dataset.map(extract_text, remove_columns=dataset.column_names)
dataset

  0%|          | 0/1892723 [00:00<?, ?ex/s]

Dataset({
    features: ['text'],
    num_rows: 1892723
})

Great. Our dataset is already finished. Let's upload it to the Hub so that we can inspect and reuse it better.

You can log in by executing the following cell.

Next, we call 🤗 Hugging Face's [`push_to_hub`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=push#datasets.Dataset.push_to_hub) method to upload the dataset to the repo `"swedish_corpora_parliament_processed"`.

In [ ]:
dataset.push_to_hub(f"{target_lang}_corpora_parliament_processed", split="train")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

That was easy! The dataset viewer is automatically enabled when uploading a new dataset, which is very convenient. You can now directly inspect the dataset online. 

Feel free to look through our preprocessed dataset directly on [`hf-test/sv_corpora_parliament_processed`](https://huggingface.co/datasets/hf-test/sv_corpora_parliament_processed). Even if we are not a native speaker in Swedish, we can see that the data is well processed and seems clean.

Next, let's use the data to build a language model.

## **3. Build an *n-gram* with KenLM**

While large language models based on the [Transformer architecture](https://jalammar.github.io/illustrated-transformer/) have become the standard in NLP, it is still very common to use an ***n-gram*** LM to boost speech recognition systems - as shown in Section 1.

Looking again at Table 9 of Appendix C of the [official Wav2Vec2 paper](https://arxiv.org/abs/2006.11477), it can be noticed that using a *Transformer*-based LM for decoding clearly yields better results than using an *n-gram* model, but the difference between *n-gram* and *Transformer*-based LM is much less significant than the difference between *n-gram* and no LM. 

*E.g.*, for the large Wav2Vec2 checkpoint that was fine-tuned on 10min only, an *n-gram* reduces the word error rate (WER) compared to no LM by *ca.* 80% while a *Transformer*-based LM *only* reduces the WER by another 23% compared to the *n-gram*. This relative WER reduction becomes less, the more data the acoustic model has been trained on. *E.g.*, for the large checkpoint a *Transformer*-based LM reduces the WER by merely 8% compared to an *n-gram* LM whereas the *n-gram* still yields a 21% WER reduction compared to no language model.

The reason why an *n-gram* is preferred over a *Transformer*-based LM is that *n-grams* come at a significantly smaller computational cost. For an *n-gram*, retrieving the probability of a word given previous words is almost only as computationally expensive as querying a look-up table or tree-like data storage - *i.e.* it's very fast compared to modern *Transformer*-based language models that would require a full forward pass to retrieve the next word probabilities.

For more information on how *n-grams* function and why they are (still) so useful for speech recognition, the reader is advised to take a look at [this excellent summary](https://web.stanford.edu/~jurafsky/slp3/3.pdf) from Stanford.

Great, let's see step-by-step how to build an *n-gram*. We will use the popular [KenLM library](https://github.com/kpu/kenlm) to do so. Let's start by installing the Ubuntu library prerequisites:

In [ ]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-program-options-dev is already the newest version (1.71.0.0ubuntu2).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.71.0.0ubuntu2).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.71.0.0ubuntu2).
libboost-thread-dev set to manually installed.
libbz2-dev is already the newest version (1.0.8-2).
libbz2-dev set to manually installed.
libboost-test-dev is already the newest version (1.71.0.0ubuntu2).
libboost-test-dev set to manually installed.
build-essential is already the newest version (12.8ubuntu1.1).
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
liblzma-dev is already the newest version (5.2.4-1ubuntu1.1).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu1.5).
zlib1g-dev set to manually installed.
Su

before downloading and unpacking the KenLM repo.

In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2023-03-05 03:36:03--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  1.66MB/s    in 0.3s    

2023-03-05 03:36:03 (1.66 MB/s) - written to stdout [491888/491888]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.71.0/BoostConfig.cmake (found suitable version "1.71.0", minimum required is "1.41.0") found components: program_options system thread unit_test_framework 
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found version "1.2.11") 
-- Found BZip2: /usr/lib/x86_64-linux-gnu/libbz2.so (found version "1.0.8") 
-- Looking for BZ2_bzCompr

Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

In [ ]:
from datasets import load_dataset

username = "yip-i"  # change to your username

dataset = load_dataset(f"{username}/{target_lang}_corpora_parliament_processed", split="train")

with open("text.txt", "w") as file:
  file.write(" ".join(dataset["text"]))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1892723 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/yip-i___parquet/yip-i--en_corpora_parliament_processed-85054519067dd648/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built 
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [ ]:

!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 46559472 types 157237
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1886844 2:1062553984 3:1992288896 4:3187662080 5:4648674304
Statistics:
1 157236 D1=0.695418 D2=0.971506 D3+=1.28866
2 3607423 D1=0.719186 D2=1.06157 D3+=1.37433
3 15224680 D1=0.80515 D2=1.12215 D3+=1.35694
4 28995341 D1=0.883552 D2=1.19479 D3+=1.37918
5 38115509 D1=0.920847 D2=1.23521 D3+=1.33938
Memory estimate for binary LM:
type      MB
probing 1752 assuming -p 1.5
probing 2027 assuming -r models -p 1.5
trie     833 without quantization
trie     461 assuming -q 8 -b 8 quantization 
trie     731 assuming -a 22 array pointer compression
trie     359 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [ ]:
!head -20 5gram.arpa

\data\
ngram 1=157236
ngram 2=3607423
ngram 3=15224680
ngram 4=28995341
ngram 5=38115509

\1-grams:
-6.5903764	<unk>	0
0	<s>	-0.14315866
-4.558773	resumption	-0.95883846
-2.2413182	of	-1.0859153
-2.0961094	the	-1.2426006
-4.1075063	session	-0.58734393
-2.3016198	i	-1.7456352
-4.4685726	declare	-0.42787033
-4.76735	resumed	-0.35256708
-2.9149663	european	-0.82456565
-3.0663066	parliament	-0.7743336
-5.264105	adjourned	-0.3560296


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [ ]:
with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in read_file:
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

Let's now inspect the corrected *5-gram*.

In [ ]:
!head -20 5gram_correct.arpa

\data\
ngram 1=157237
ngram 2=3607423
ngram 3=15224680
ngram 4=28995341
ngram 5=38115509

\1-grams:
-6.5903764	<unk>	0
0	<s>	-0.14315866
0	</s>	-0.14315866
-4.558773	resumption	-0.95883846
-2.2413182	of	-1.0859153
-2.0961094	the	-1.2426006
-4.1075063	session	-0.58734393
-2.3016198	i	-1.7456352
-4.4685726	declare	-0.42787033
-4.76735	resumed	-0.35256708
-2.9149663	european	-0.82456565
-3.0663066	parliament	-0.7743336


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

## **4. Combine an *n-gram* with Wav2Vec2**

In a final step, we want to wrap the *5-gram* into a `Wav2Vec2ProcessorWithLM` object to make the *5-gram* boosted decoding as seamless as shown in Section 1.
We start by downloading the currently "LM-less" processor of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv).

In [ ]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Next, we extract the vocabulary of its tokenizer as it represents the `"labels"` of `pyctcdecode`'s `BeamSearchDecoder` class.

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

The `"labels"` and the previously built `5gram_correct.arpa` file is all that's needed to build the decoder. 

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

We can safely ignore the warning and all that is left to do now is to wrap the just created `decoder`, together with the processor's `tokenizer` and `feature_extractor` into a `Wav2Vec2ProcessorWithLM` class.

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

We want to directly upload the LM-boosted processor into 
the model folder of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv) to have all relevant files in one place.

Let's clone the repo, add the new decoder files and upload them afterward.
First, we need to install `git-lfs`.

In [ ]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 0s (155 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ..

Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class. 

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).

In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir="model_staging", clone_from=model_name)

Cloning https://huggingface.co/yip-i/torgo_xlsr_finetune-F04-2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/1.18G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##9       | 1.00k/3.43k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Having cloned `xls-r-300m-sv`, let's save the new processor with LM into it.

In [ ]:
processor_with_lm.save_pretrained("model_staging")

Let's inspect the local repository. The `tree` command conveniently can also show the size of the different files.

In [ ]:
!tree -h model_staging/

model_staging/
├── [  30]  added_tokens.json
├── [ 198]  alphabet.json
├── [2.3K]  config.json
├── [4.0K]  language_model
│   ├── [3.6G]  5gram_correct.arpa
│   ├── [  78]  attrs.json
│   └── [1.5M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [3.3K]  README.md
├── [ 406]  special_tokens_map.json
├── [ 569]  tokenizer_config.json
├── [3.4K]  training_args.bin
└── [ 331]  vocab.json

1 directory, 13 files


As can be seen the *5-gram* LM is quite large - it amounts to more than 4 GB.
To reduce the size of the *n-gram* and make loading faster, `kenLM` allows converting `.arpa` files to binary ones using the `build_binary` executable.

Let's make use of it here.

In [ ]:
!kenlm/build/bin/build_binary model_staging/language_model/5gram_correct.arpa model_staging/language_model/5gram.bin

Reading model_staging/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


Great, it worked! Let's remove the `.arpa` file and check the size of the binary *5-gram* LM.

In [ ]:
!rm model_staging/language_model/5gram_correct.arpa && tree -h model_staging/

model_staging/
├── [  30]  added_tokens.json
├── [ 198]  alphabet.json
├── [2.3K]  config.json
├── [4.0K]  language_model
│   ├── [1.7G]  5gram.bin
│   ├── [  78]  attrs.json
│   └── [1.5M]  unigrams.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [3.3K]  README.md
├── [ 406]  special_tokens_map.json
├── [ 569]  tokenizer_config.json
├── [3.4K]  training_args.bin
└── [ 331]  vocab.json

1 directory, 13 files


Nice, we reduced the *n-gram* by more than half to less than 2GB now. In the final step, let's upload all files.

In [ ]:
repo.push_to_hub(commit_message="Upload lm-boosted decoder")

Upload file language_model/5gram.bin:   0%|          | 32.0k/1.71G [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/yip-i/torgo_xlsr_finetune-F04-2
   9b48c0d..7d6a87a  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/yip-i/torgo_xlsr_finetune-F04-2
   9b48c0d..7d6a87a  main -> main



'https://huggingface.co/yip-i/torgo_xlsr_finetune-F04-2/commit/7d6a87ab9b9ffb600e976add2134723c2f201cf5'

## **5.Evaluation**

In [ ]:
%cd /content
!gdown 1TaP_UX2QiiKR_qWMeYtlMgHaXlYRliJ9
!mkdir downloads
%cd downloads
!gdown 1ZF4gQDnq8gHW4VRXJa5m3yIRhy5qcJmp
!tar -xzf torgo.tar.gz && ls torgo

/content
Downloading...
From: https://drive.google.com/uc?id=1TaP_UX2QiiKR_qWMeYtlMgHaXlYRliJ9
To: /content/output.csv
100% 607k/607k [00:00<00:00, 108MB/s]
/content/downloads
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1ZF4gQDnq8gHW4VRXJa5m3yIRhy5qcJmp 

tar (child): torgo.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
!cp -r /content/drive/MyDrive/Northeastern\ University/Research\ Assisstant/Torgo /content/downloads/Torgo

/content/downloads


### Processing Data

In [ ]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset, Audio

data = load_dataset('csv', data_files='/content/output.csv')
data = data.cast_column("audio", Audio(sampling_rate=16_000))
timit = data['train'].filter(lambda x: x == speaker, input_columns=['speaker_id'])


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2ProcessorWithLM.from_pretrained("/content/model_staging")
model = Wav2Vec2ForCTC.from_pretrained(model_name)

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
timit = timit.map(remove_special_characters)

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        # batch["labels"] = processor(batch["text"]).input_ids
        batch["labels"] = batch["text"]
        # print(processor(batch["text"]))
    return batch

In [ ]:
timit = timit.map(prepare_dataset, remove_columns=timit.column_names, num_proc=4)
timit = timit.filter(lambda x: x < 25 * processor.feature_extractor.sampling_rate, input_columns=["input_length"])


#0:   0%|          | 0/167 [00:00<?, ?ex/s]

#1:   0%|          | 0/167 [00:00<?, ?ex/s]

#2:   0%|          | 0/167 [00:00<?, ?ex/s]

#3:   0%|          | 0/166 [00:00<?, ?ex/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:543: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:543: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:543: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by usi

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cpu").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [ ]:
timit[0]

{'input_values': [-0.5296888947486877,
  -0.06537819653749466,
  0.00725141353905201,
  -0.6075063347816467,
  0.05134795978665352,
  -0.37405404448509216,
  -0.4622471332550049,
  0.012439243495464325,
  -0.41036882996559143,
  0.03578447550535202,
  -0.13022606074810028,
  -0.02906339056789875,
  -0.10428691655397415,
  0.10582016408443451,
  -0.11725649237632751,
  0.07728710770606995,
  0.11619582772254944,
  0.03578447550535202,
  0.01762707158923149,
  -0.07834777235984802,
  -0.11985040456056595,
  0.00725141353905201,
  0.42487165331840515,
  -0.6049124598503113,
  0.26923680305480957,
  0.05134795978665352,
  -0.01090598851442337,
  0.14213496446609497,
  0.13175931572914124,
  0.12397757172584534,
  -0.22879481315612793,
  0.30555158853530884,
  0.24589155614376068,
  -0.17691652476787567,
  0.32889682054519653,
  -0.07834777235984802,
  0.2951759397983551,
  0.02540881559252739,
  0.11878973990678787,
  0.3107394278049469,
  -0.10428691655397415,
  0.34446030855178833,
  -0.

In [ ]:
import torch

def get_result(torgo_dataset):
  pred_str = []
  actual = []
  for i in range(torgo_dataset.num_rows):
    inputs = processor(torgo_dataset[i]["input_values"], sampling_rate=16_000, return_tensors="pt")
    with torch.no_grad():
      logits = model(**inputs).logits
    transcription = processor.batch_decode(logits.numpy()).text
    pred_str.append(transcription[0].lower())
    actual = processor.decode(torgo_dataset[i]["labels"]).text

  return pred_str, actual
    

In [ ]:
inputs = processor(timit[0]["input_values"], sampling_rate=16_000, return_tensors="pt")

In [ ]:

with torch.no_grad():
  logits = model(**inputs).logits

In [ ]:
logits.shape

torch.Size([1, 142, 32])

In [ ]:
transcription = processor.batch_decode(logits.numpy()).text
transcription[0].lower()

'fee'

In [ ]:
from tqdm import tqdm

pred_str = []
actual = []
for i in tqdm(range(timit.num_rows)):
  inputs = processor(timit[i]["input_values"], sampling_rate=16_000, return_tensors="pt")
  with torch.no_grad():
    logits = model(**inputs).logits
  transcription = processor.batch_decode(logits.numpy()).text
  pred_str.append(transcription[0].lower())

  actual.append(timit[i]["labels"])




100%|██████████| 667/667 [45:07<00:00,  4.06s/it]


In [ ]:
# results = timit.map(map_to_result, remove_columns=timit.column_names, batch_size=1, writer_batch_size=1, load_from_cache_file=False)
# pred_str, actual = get_result(timit)

AttributeError: ignored

In [ ]:
wer_metric = load_metric("wer")
wer_metric.compute(predictions = pred_str, references = actual)
# print("Test WER: {:.3f}".format(wer_metric.compute(pred_str, actual))

0.07294117647058823